In [ ]:
import pandas as pd

In [ ]:
documents = pd.read_csv("/Users/martinkvammen/Google Drive (martin.kvammen@gmail.com)/1. Masters Thesis/5. Shared Thesis /3.1. Data/1. Data processing /Preprocessed data/preprocessed_original_easyjet.csv")
documents.head()

,Unnamed: 0,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,...,National Viewership,Engagement,AVE,Sentiment,Key Phrases,Input Name,Keywords,Document Tags,index,Processed
0,0,01-Aug-2019 01:00AM,NaN,http://twitter.com/RageSnow_z/statuses/1156701...,NaN,=@bell_allie @SouthendAirport @easyJet Probabl...,Twitter,=@RageSnow_z,United Kingdom,NaN,...,0,0.0,1.30,Neutral,scheduled flight,Adhoc Search Export,easyJet,NaN,0,"['bell_alli', 'southendairport', 'easyjet', 'p..."
1,1,01-Aug-2019 01:01AM,NaN,https://twitter.com/WalterO32308411/statuses/1...,NaN,@RGrosjean @easyJet Go Pietro Go!!!____,Twitter,=@WalterO32308411,Unknown,NaN,...,0,NaN,0.00,Negative,NaN,Adhoc Search Export,easyJet,NaN,1,"['rgrosjean', 'easyjet', 'pietro']"
2,2,01-Aug-2019 01:01PM,NaN,https://twitter.com/OliRibbbbbs/statuses/11568...,NaN,@easyJet why does your submission form on your...,Twitter,=@OliRibbbbbs,Unknown,NaN,...,0,NaN,0.00,Negative,"can't get hold,customer services,delivery,emai...",Adhoc Search Export,easyJet,NaN,2,"['easyjet', 'submiss', 'form', 'websit', 'work..."
3,3,01-Aug-2019 01:01PM,NaN,http://twitter.com/capreoli94/statuses/1156882...,NaN,"@Gatwick_Airport Hi, my flight to Pisa, with E...",Twitter,=@capreoli94,Italy,NaN,...,0,3.0,1.54,Negative,"canceled flight,serious problem,kind,hours,Eas...",Adhoc Search Export,Easyjet,NaN,3,"['gatwick_airport', 'flight', 'pisa', 'easyjet..."
4,4,01-Aug-2019 01:02AM,NaN,https://twitter.com/Bagsy45/statuses/115670172...,NaN,=@brothers_beyond @easyJet Hi @brothers_beyond...,Twitter,=@Bagsy45,Unknown,NaN,...,0,NaN,0.00,Negative,"flight,reasons for cancellation,screen shots,w...",Adhoc Search Export,"easyJet,easyjet",NaN,4,"['brothers_beyond', 'easyjet', 'brothers_beyon..."


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikolajfrandsen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
processed_docs = documents['Processed'].map(preprocess)

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 bell_al
1 easyjet
2 flight
3 probabl
4 schedul
5 southendairport
6 pietro
7 rgrosjean
8 custom
9 deliveri
10 email


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(41, 1), (44, 1), (89, 1), (1048, 1), (1053, 1), (1542, 1)]

In [ ]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 41 ("time") appears 1 time.
Word 44 ("thank") appears 1 time.
Word 89 ("today") appears 1 time.
Word 1048 ("special") appears 1 time.
Word 1053 ("super") appears 1 time.
Word 1542 ("biarritz") appears 1 time.


In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4644317948159989), (1, 0.517480014730821), (2, 0.7186915488011385)]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.010*"rgrosjean" + 0.010*"time" + 0.008*"custom" + 0.007*"servic" + 0.007*"pictur" + 0.007*"good" + 0.007*"airlin" + 0.007*"delay" + 0.006*"like" + 0.006*"passeng"
Topic: 1 
Words: 0.050*"spare" + 0.046*"real" + 0.044*"absolut" + 0.042*"current" + 0.041*"photo" + 0.039*"plane" + 0.039*"decend" + 0.017*"stewardess" + 0.015*"passeng" + 0.015*"julianstorey"
Topic: 2 
Words: 0.008*"passeng" + 0.007*"safeti" + 0.006*"luggag" + 0.006*"board" + 0.006*"crew" + 0.006*"like" + 0.006*"child" + 0.006*"need" + 0.005*"aircraft" + 0.005*"control"
Topic: 3 
Words: 0.098*"beat" + 0.096*"allow" + 0.096*"luton" + 0.094*"geneva" + 0.093*"backless" + 0.086*"geneveaeroport" + 0.085*"iata" + 0.085*"easa" + 0.085*"easyjet_press" + 0.083*"ryanair"
Topic: 4 
Words: 0.148*"abditv" + 0.147*"canõt" + 0.145*"believ" + 0.144*"explain" + 0.029*"upto" + 0.018*"compen" + 0.016*"person" + 0.015*"stansaidairport" + 0.013*"sure" + 0.013*"check"
Topic: 5 
Words: 0.098*"flylo" + 0.054*"omar" + 0.054*"baba"

In [ ]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=bow_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/Users/nikolajfrandsen/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
processed_docs[4310]

['micou',
 'special',
 'guest',
 'today',
 'biarritz',
 'flight',
 'thank',
 'alafpolak',
 'super',
 'time',
 'easyjet']

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8713957667350769	 
Topic: 0.022*"delay" + 0.014*"board" + 0.012*"thank" + 0.011*"holiday" + 0.011*"hour" + 0.011*"compen" + 0.011*"tell" + 0.010*"wait" + 0.010*"airlin" + 0.009*"like"

Score: 0.014294627122581005	 
Topic: 0.095*"thank" + 0.093*"remov" + 0.087*"ross" + 0.081*"info" + 0.076*"matthew" + 0.073*"best" + 0.071*"photograph" + 0.071*"assist" + 0.070*"bring" + 0.070*"investig"

Score: 0.014291099272668362	 
Topic: 0.026*"rgrosjean" + 0.018*"hope" + 0.016*"good" + 0.015*"attend" + 0.015*"aviat" + 0.014*"video" + 0.013*"airbus" + 0.013*"need" + 0.013*"peopl" + 0.012*"soon"

Score: 0.014290914870798588	 
Topic: 0.034*"cancel" + 0.032*"delay" + 0.025*"hour" + 0.021*"airport" + 0.017*"servic" + 0.015*"claim" + 0.015*"custom" + 0.014*"time" + 0.010*"compen" + 0.009*"book"

Score: 0.014289424754679203	 
Topic: 0.031*"sure" + 0.022*"work" + 0.020*"know" + 0.019*"time" + 0.019*"rout" + 0.015*"children" + 0.014*"tweet" + 0.014*"thing" + 0.013*"think" + 0.013*"break"

Score: 0.0